In [1]:
#最低限必要な情報:物件名・住所・間取り・家賃・階数
#希望条件：都内で利便性の高い場所・予算20万円程度・2人暮らし2K以上
#検索条件：JR山手線沿線・予算25万円以下・2K以上の間取り

In [2]:
# STEP１：全ページの物件情報を全て取ってくる

# 必要なライブラリのインポート
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
from tqdm import tqdm #for文の進捗確認

# 最後のページの数値を取得する
root_url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?fw2=&mt=9999999&cn=9999999&co=1&ra=013&et=9999999&shkr1=03&ar=030&bs=040&ct=25.0&shkr3=03&shkr2=03&mb=0&md=05&md=06&md=07&md=08&md=09&md=10&md=11&md=12&md=13&md=14&rn=0005&shkr4=03&cb=0.0&page={}'
res = requests.get(root_url.format(1))
res.encoding = 'utf-8'
soup = BeautifulSoup(res.text, 'html.parser')
total_page = int(soup.find('ol', class_='pagination-parts').find_all('li')[-1].text)

#物件情報を1件ずつ格納するための空のリストを用意しておく
d_list = []

# 正常にHTMLの情報が取得できれば以下のコードを実行
if res.status_code == 200:

    # 物件一覧ページをfor文で1ページずつループ
    for i in tqdm(range(1, total_page+1)):
        root_url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?fw2=&mt=9999999&cn=9999999&co=1&ra=013&et=9999999&shkr1=03&ar=030&bs=040&ct=25.0&shkr3=03&shkr2=03&mb=0&md=05&md=06&md=07&md=08&md=09&md=10&md=11&md=12&md=13&md=14&rn=0005&shkr4=03&cb=0.0&page={}'
        res = requests.get(root_url.format(i))
        res.encoding = 'utf-8'
        soup = BeautifulSoup(res.text, 'html.parser')
        time.sleep(1)

        # 物件一覧ページの物件情報をfor文で1件ずつループ
        contents = soup.find_all('div', class_='cassetteitem')
        for content in contents:

            # 物件情報をdetailに格納する
            detail = content.find('div', class_='cassetteitem-detail')
            # 物件名
            name = detail.find('div', class_='cassetteitem_content-title').text
            # 住所
            address = detail.find('li', class_='cassetteitem_detail-col1').text
            # アクセス
            access1 = detail.find_all('div', class_='cassetteitem_detail-text')[0].text
            access2 = detail.find_all('div', class_='cassetteitem_detail-text')[1].text
            access3 = detail.find_all('div', class_='cassetteitem_detail-text')[2].text
            # 築年数/構造
            age = detail.find('li', class_='cassetteitem_detail-col3').find_all('div')[0].text
            structure = detail.find('li', class_='cassetteitem_detail-col3').find_all('div')[1].text
            # 物件画像リンク
            try:
                build_img_url = detail.find('img', class_='js-linkImage').get('rel')
            except Exception as e:
                build_img_url = None

            # 物件情報の中で、部屋情報をfor文で1件ずつループ
            table = content.find('table', class_='cassetteitem_other')
            tr_tags = table.find_all('tr', class_='js-cassette_link')
            for tr_tag in tr_tags:

                # 部屋情報
                floor, price, first_fee, capacity = tr_tag.find_all('td')[2:6]
                floor = floor.text.replace('\r','').replace('\n','').replace('\t','')
                rent, admin = price.find_all('li')
                deposit, gratuity = first_fee.find_all('li')
                madori, menseki = capacity.find_all('li')
                madori_img_url = tr_tags[0].find_all('td')[1].find('img').get('rel')
                detail_url = 'https://suumo.jp' + tr_tag.find_all('td')[8].find('a').get('href')

                # １つの部屋情報をまとめて辞書dに格納
                d = {
                    'name':name,
                    'address':address,
                    'access1':access1,
                    'access2':access2,
                    'access3':access3,
                    'age':age,
                    'structure':structure,
                    'build_img_url':build_img_url,
                    'floor':floor,
                    'rent':rent.text,
                    'admin':admin.text,
                    'deposit':deposit.text,
                    'gratuity':gratuity.text,
                    'madori':madori.text,
                    'menseki':menseki.text,
                    'madori_img_url':madori_img_url,
                    'detail_url':detail_url,
                }

                # 部屋情報をd_listに1件ずつ追加
                d_list.append(d)

#完成したリストはDataFrameに格納
df = pd.DataFrame(d_list)

100%|██████████| 74/74 [02:59<00:00,  2.42s/it]


In [3]:
#STEP2：データクレンジング

#アクセスから路線・駅名・徒歩時間を抽出。.str.split()で分割しても良い。
df[['access1_line', 'access1_station', 'access1_walk']] = df['access1'].str.extract(r'(.+?)/(.+?) 歩(.+?)分')
df[['access2_line', 'access2_station', 'access2_walk']] = df['access2'].str.extract(r'(.+?)/(.+?) 歩(.+?)分')
df[['access3_line', 'access3_station', 'access3_walk']] = df['access3'].str.extract(r'(.+?)/(.+?) 歩(.+?)分')

# addressカラムの各要素に対して末尾が1〜9であれば「丁目」を結合する。後々geopyを適用して、緯度経度の情報を取得するために必要。
# df['address'] = df['address'].apply(lambda x: x + '丁目' if x[-1] in '１２３４５６７８９' else x)

#ageを質的データから量的データ（比例尺度）に変換。99年以上は99に集約。
df['age'] = df['age'].str.replace('新築', '築0年').str.replace('築','').str.replace('年','').str.replace('以上','').astype(int)

#お金（rent, admin, deposit, gratuity）のデータ（比例尺度）に変換
def extract_amount(text):
    try:
        if '-' in text:
            amount = 0
        elif '万円' in text:
            # '万円' を削除して、float型に変換してから10000倍する
            amount = float(text.replace('万円', '')) * 10000
        else:
            # '円' を削除して、float型に変換
            amount = float(text.replace('円', ''))
    except ValueError:
        amount = None
    return int(amount)
df['rent'] = df['rent'].apply(extract_amount)
df['admin'] = df['admin'].apply(extract_amount)
df['deposit'] = df['deposit'].apply(extract_amount)
df['gratuity'] = df['gratuity'].apply(extract_amount)

#mensekiのデータ（比例尺度）に変換
df['menseki'] = df['menseki'].str.replace('m2','').astype(float)

#structure, floorのデータの表現を統一
df['structure'] = df['structure'].str.replace('地下', 'B').str.replace('地上', '-').str.replace('平屋', '1階建').str.replace('階建', 'F')
df['floor'] = df['floor'].str.replace('階', '')

#欠損値を'-'で埋める
df = df.fillna('-')

#不要なカラムを削除
df = df.drop(columns=['access1','access2','access3'])

#name, address, floor, rent, madoriが一致する物件を重複排除
df = df.drop_duplicates(subset=['name','address', 'floor', 'rent', 'madori'])

In [4]:
!pip install urllib

ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matching distribution found for urllib


In [7]:
#STEP3:国土地理院APIで住所から緯度経度の情報を取得し、dfに結合
import pandas as pd
import requests
import urllib
import folium
import tqdm

# 国土地理院API
GeospatialUrl = "https://msearch.gsi.go.jp/address-search/AddressSearch?q="

# 国土地理院APIより住所→緯度経度に変換
lat_list = []
lng_list = []
for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0]):
    s_quote = urllib.parse.quote(row.address)
    response = requests.get(GeospatialUrl + s_quote)
    try:
        lat_list.append(response.json()[0]["geometry"]["coordinates"][1])
        lng_list.append(response.json()[0]["geometry"]["coordinates"][0])
    except Exception as e:
        print(e)

# inputデータに緯度経度を追加する
df_new = df.copy()
try:
    df_new['lat'] = lat_list
    df_new['lng'] = lng_list
except Exception as e:
    print(e)


100%|██████████| 3008/3008 [04:13<00:00, 11.88it/s]


In [8]:
df_new.head()

,name,address,age,structure,build_img_url,floor,rent,admin,deposit,gratuity,...,access1_station,access1_walk,access2_line,access2_station,access2_walk,access3_line,access3_station,access3_walk,lat,lng
0,勘五郎ビル,東京都北区田端１,34,B1-7F,https://img01.suumo.com/front/gazo/fr/bukken/7...,2,135000,5000,135000,135000,...,西日暮里駅,6,ＪＲ山手線,田端駅,7,東京メトロ千代田線,千駄木駅,10,35.734928,139.761551
1,勘五郎ビル,東京都北区田端１,34,B1-7F,https://img01.suumo.com/front/gazo/fr/bukken/7...,5,160000,5000,160000,160000,...,西日暮里駅,6,ＪＲ山手線,田端駅,7,東京メトロ千代田線,千駄木駅,10,35.734928,139.761551
2,ＪＲ山手線 田端駅 5階建 築27年,東京都北区田端３,27,5F,https://img01.suumo.com/front/gazo/fr/bukken/5...,2,133000,10000,133000,133000,...,田端駅,8,ＪＲ山手線,駒込駅,9,ＪＲ山手線,西日暮里駅,15,35.735840,139.755463
3,山口貸戸建アパート,東京都荒川区西日暮里４,47,2F,https://img01.suumo.com/front/gazo/fr/bukken/8...,1-2,95000,0,95000,95000,...,西日暮里駅,4,ＪＲ山手線,田端駅,6,ＪＲ山手線,日暮里駅,15,35.732731,139.763702
4,山口貸戸建アパート,東京都荒川区西日暮里４,47,2F,https://img01.suumo.com/front/gazo/fr/bukken/8...,1,95000,5500,95000,95000,...,西日暮里駅,4,ＪＲ山手線,田端駅,6,ＪＲ山手線,日暮里駅,15,35.732731,139.763702


In [11]:
df_new.isnull().any()

name               False
address            False
age                False
structure          False
build_img_url      False
floor              False
rent               False
admin              False
deposit            False
gratuity           False
madori             False
menseki            False
madori_img_url     False
detail_url         False
access1_line       False
access1_station    False
access1_walk       False
access2_line       False
access2_station    False
access2_walk       False
access3_line       False
access3_station    False
access3_walk       False
lat                False
lng                False
dtype: bool

In [13]:
# STEP4：dfをRDBのSQLiteに書き込む
#必要なライブラリのインポート
import sqlite3
# databaseに接続する。指定したdatabaseが存在しない場合は、新規作成される。
conn = sqlite3.connect('suumo_v3.db')
# 物件情報が入ったdfをsqlに書き出す。
df_new.to_sql('suumo_v3', conn, if_exists='replace', index=False)

# databaseをクエリで実行した結果を読み込み、df_from_sqliteに代入。printで書き込み結果を確認する。
query = "SELECT * FROM suumo_v3"
df_from_sqlite = pd.read_sql(query, conn)
print(df_from_sqlite)

#databaseとの接続を終了
conn.close()

                    name      address  age structure  \
0                  勘五郎ビル     東京都北区田端１   34     B1-7F   
1                  勘五郎ビル     東京都北区田端１   34     B1-7F   
2     ＪＲ山手線 田端駅 5階建 築27年     東京都北区田端３   27        5F   
3              山口貸戸建アパート  東京都荒川区西日暮里４   47        2F   
4              山口貸戸建アパート  東京都荒川区西日暮里４   47        2F   
...                  ...          ...  ...       ...   
3003    ＪＲ山手線 池袋駅 3階建 新築   東京都豊島区南池袋３    0        3F   
3004    ＪＲ山手線 池袋駅 3階建 新築   東京都豊島区南池袋３    0        3F   
3005         Ｈａｌｅ　Ｎａｎａｌａ   東京都豊島区南池袋３    0        3F   
3006         Ｈａｌｅ　Ｎａｎａｌａ   東京都豊島区南池袋３    0        3F   
3007           アルカディア恵比寿   東京都渋谷区恵比寿３    0        3F   

                                          build_img_url floor    rent  admin  \
0     https://img01.suumo.com/front/gazo/fr/bukken/7...     2  135000   5000   
1     https://img01.suumo.com/front/gazo/fr/bukken/7...     5  160000   5000   
2     https://img01.suumo.com/front/gazo/fr/bukken/5...     2  133000  10000   
3     h

In [ ]:
!pip install streamlit --upgrade